### Dépendances

In [95]:
import urllib
import bs4
import pandas as pd
from urllib import request
import numpy as np
import re

**Objectif final:** créer le dataframe composé des auteurs et de leur mouvement littéraire.

### Scraping de la page recensant les mouvements littéraires

In [107]:
mvm_url="https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_par_mouvement_ou_courant_litt%C3%A9raire"
mvm_request=request.urlopen(mvm_url).read()
mvm_page = bs4.BeautifulSoup(mvm_request, "lxml") #qui est bien une page html maintenant 

<code>mvm_blayer</code> ; on identifie le bloc regroupant tous les mouvements avec <code>id</code>.<br> 
Son premier élément est HS, on le retire.

In [108]:
mvm_blayer=mvm_page.find("div",{"id":"mw-subcategories"}).findAll("ul")
mvm_blayer=mvm_blayer[1::]

<code>mvm_list</code> liste issue du scraping de la page précédente : composée de l'ensemble des mouvements littéraires de la forme 'Catégorie:XXX'. Cette forme sera utile pour créer les URLs.<br>
<code>data_mvm</code> liste précédente nettoyée de sorte à n'avoir que le nom des mouvements.

In [110]:
mvm_list=[]
for ul in mvm_blayer:
    interli=[]
    listli=ul.findAll("li")
    for li in listli:
        try:
            interli.append(li.find("a").get("title"))
        except:
            pass
    mvm_list.append(interli)
#print(mvm_list)

# Liste des mouvements qui sera utilisée pour créer le dataframe 
data_mvm=[]
for l in mvm_list:
    for el in l:
        pep=re.sub('Catégorie:','',el)
        data_mvm.append(pep)
data_mvm

['Écrivain de la Beat Generation',
 'Écrivain de la Génération perdue',
 'Écrivain du courant des Hussards',
 'Écrivain jungien',
 'Écrivain du courant réaliste',
 'Écrivain régionaliste',
 'Écrivain de la Renaixença',
 'Écrivain romantique',
 'Écrivain surréaliste',
 'Écrivain symboliste']

On modifie <code>mvm_list</code> afin d'être utilisable pour les URLs simplement en concaténant une base fixe et ses éléments.

In [111]:
def modifL(list):
    L=[]
    for i in list:
        for j in i:
            pep=re.sub(' ','_',j)               # on remplace les espaces 
            pep=pep.replace('é','%C3%A9')       # corrections pour l'url
            pep=pep.replace('É','%C3%89')           
            pep=pep.replace('ç','%C3%A7')
            L.append(pep)   
    return(L)

mvm_list=modifL(mvm_list)
mvm_list

['Cat%C3%A9gorie:%C3%89crivain_de_la_Beat_Generation',
 'Cat%C3%A9gorie:%C3%89crivain_de_la_G%C3%A9n%C3%A9ration_perdue',
 'Cat%C3%A9gorie:%C3%89crivain_du_courant_des_Hussards',
 'Cat%C3%A9gorie:%C3%89crivain_jungien',
 'Cat%C3%A9gorie:%C3%89crivain_du_courant_r%C3%A9aliste',
 'Cat%C3%A9gorie:%C3%89crivain_r%C3%A9gionaliste',
 'Cat%C3%A9gorie:%C3%89crivain_de_la_Renaixen%C3%A7a',
 'Cat%C3%A9gorie:%C3%89crivain_romantique',
 'Cat%C3%A9gorie:%C3%89crivain_surr%C3%A9aliste',
 'Cat%C3%A9gorie:%C3%89crivain_symboliste']

### Scraping liste d'auteurs par mouvement 

A première vue on a deux types de page :
- Celles avec une partie "sous-catégorie" puis le bloc avec les écrivains
- Celles sans "sous-catégorie"<br>
#
**Solution** : passer par l'identification <code>.find("div",{"id":'mw-pages'})</code>

Maintenant qu'on a la liste des différents mouvements on veut une liste d'auteurs par mouvement.<br>
On va scraper les pages associées à ces mouvements de la forme <code>https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:XXX</code> (où XXX est un mouvement). 

In [92]:
# == Création des URLs ==
wikibase="https://fr.wikipedia.org/wiki/"

url_list=[wikibase+cat for cat in mvm_list]
url_list

['https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_de_la_Beat_Generation',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_de_la_G%C3%A9n%C3%A9ration_perdue',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_du_courant_des_Hussards',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_jungien',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_du_courant_r%C3%A9aliste',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_r%C3%A9gionaliste',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_de_la_Renaixen%C3%A7a',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_romantique',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_surr%C3%A9aliste',
 'https://fr.wikipedia.org/wiki/Cat%C3%A9gorie:%C3%89crivain_symboliste']

<code>aut_crap</code> fonction qui prend en argument un URL et retourne la liste des auteurs présents sur cette page.

In [93]:
def aut_scrap(url):

    request_text= request.urlopen(url).read()
    page = bs4.BeautifulSoup(request_text, "lxml") #qui est bien une page html maintenant 

    # == Obtenir toutes les catégories par lettre ==
    base=page.find("div",{"id":'mw-pages'}).findAll("ul")
    base=base[1::]

    # == Obtenir les auteurs pour tous les bloc précédents ==
    aut_list=[]
    auteurs1=[]
    for ul in base:
        liste_li=ul.findAll("li")
        for li in liste_li:
            try:
                auteurs1.append(li.find("a").get("title"))
            except:
                pass
        
    return(auteurs1)

In [82]:
aut_scrap(url_list[6])

['Marià Aguiló',
 "Pere d'Alcàntara Penya",
 'Joan Alcover',
 'Bonaventura Carles Aribau',
 'Francesc Badenes',
 'Víctor Balaguer',
 'Antoni de Bofarull',
 'Carles Bosch de la Trinxeria',
 'Bartomeu Ferrà',
 'Àngel Guimerà',
 'Constantí Llombart',
 'Teodor Llorente Olivares',
 'Joan Maragall',
 'Manuel Milà i Fontanals',
 'Mateu Obrador',
 'Narcís Oller',
 'Josep Lluís Pons',
 'Josep Maria Puig i Torralva',
 'Vicent Wenceslau Querol',
 'Jeroni Rosselló',
 'Joaquim Rubió i Ors',
 'Josep Maria de Sagarra',
 'Josep Torras i Bages',
 'Jacint Verdaguer',
 'Tomàs Villarroya i Sanz']

<code>aut1</code> liste des auteurs groupés par mouvements.

In [112]:
aut1=[ aut_scrap(url) for url in url_list]
aut1    

[['William S. Burroughs',
  'Lucien Carr',
  'Carolyn Cassady',
  'John Clellon Holmes',
  'Gregory Corso',
  'Lenore Kandel',
  'Jack Kerouac',
  'Jan Kerouac',
  'Gerald Nicosia',
  'Peter Orlovsky',
  'John Wieners'],
 ['Sherwood Anderson',
  'Sylvia Beach',
  'E. E. Cummings',
  'John Dos Passos',
  'T. S. Eliot',
  'F. Scott Fitzgerald',
  'Ernest Hemingway',
  'Archibald MacLeish',
  'Ezra Pound',
  'Gertrude Stein',
  'John Steinbeck'],
 ['Antoine Blondin',
  'Jean-François Coulomb',
  'Michel Déon',
  'Geneviève Dormann',
  'Guy Dupré',
  'André Fraigneau',
  'Bernard Frank (écrivain)',
  'Kléber Haedens',
  'Stephen Hecquet',
  'Roland Laudenbach',
  'Jacques Laurent',
  'Félicien Marceau',
  'Roger Nimier',
  'François Nourissier',
  'Jacques Perret (écrivain)',
  'Willy de Spens'],
 ['Kristine Mann',
  'Thomas Moore (psychothérapeute)',
  'Clarissa Pinkola Estés',
  'Clifton Snider',
  'Hal Stone'],
 ['Israel Aksenfeld',
  'Leopoldo Alas',
  'Willibald Alexis',
  'Honoré de 

<code>data_mvm2</code> liste de la même forme que <code>aut1</code> qui associe à chaque écrivain son mouvement.

In [97]:
data_mvm2=[]
for i in range(len(aut1)):
    inter=[]
    for j in range(len(aut1[i])):
        inter.append(data_mvm[i])
    data_mvm2.append(inter)
data_mvm2


[['Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation',
  'Écrivain de la Beat Generation'],
 ['Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue',
  'Écrivain de la Génération perdue'],
 ['Écrivain du courant des Hussards',
  'Écrivain du courant des Hussards',
  'Écrivain du courant des Hussards',
  'Écrivain du courant des Hussards',
  'Écrivain du courant des Hussa

### Création du dataframe

In [105]:
xdata, ydata=[],[]

for gr_aut in aut1:
    for aut in gr_aut:
        xdata.append(aut)
for gr_mvm in data_mvm2:
    for mvm in gr_mvm:
        ydata.append(mvm)

data={'Nom':xdata, 'Mouvement':ydata}
df=pd.DataFrame(data)
print(df)

                      Nom                       Mouvement
0    William S. Burroughs  Écrivain de la Beat Generation
1             Lucien Carr  Écrivain de la Beat Generation
2         Carolyn Cassady  Écrivain de la Beat Generation
3     John Clellon Holmes  Écrivain de la Beat Generation
4           Gregory Corso  Écrivain de la Beat Generation
..                    ...                             ...
256     Maurice Beaubourg             Écrivain symboliste
257        Otokar Březina             Écrivain symboliste
258   Emanuil Popdimitrov             Écrivain symboliste
259         Marcel Schwob             Écrivain symboliste
260     Friedebert Tuglas             Écrivain symboliste

[261 rows x 2 columns]


In [104]:
compression_opts = dict(method='zip', archive_name='out.csv') 
df.to_csv('out.zip', index=False,compression=compression_opts)  